### Load data first

In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
import time
from src.preprocessing import read_image, windy_preprocessing_pipeline
from tqdm import tqdm
from datetime import datetime

SOURCE_PATH = "data/images/philippines_13.269_123.832"

img_paths = [
    os.path.join(SOURCE_PATH, img_name) for img_name in sorted(os.listdir(SOURCE_PATH)) if img_name.endswith('.png')
][:30]

img_shape = read_image(img_paths[0]).shape       # get shape of the image
print(f"Image shape: {img_shape}")
print(f"-" * 20)

time_frames = []
dbz_maps = []

for path in tqdm(img_paths, desc="Processing images and detecting storms"):
    time_frames.append(datetime.strptime(os.path.basename(path).split('/')[0][:-4], '%Y%m%d-%H%M%S'))       # Get file name without extension
    img = read_image(path)
    dbz_maps.append(windy_preprocessing_pipeline(img))

print(f"Number of frames: {len(dbz_maps)}")

Image shape: (505, 1220, 3)
--------------------


Processing images and detecting storms: 100%|██████████| 30/30 [00:14<00:00,  2.03it/s]

Number of frames: 30


## Load model

In [2]:
from src.models import OursPrecipitationModel
from src.identification import HypothesisIdentifier

model = OursPrecipitationModel(identifier=HypothesisIdentifier())

storms_maps = []

for idx, (dbz_map, time_frame) in enumerate(tqdm(zip(dbz_maps, time_frames), total=len(dbz_maps), desc="Detecting storms")):
    storms_map = model.identify_storms(dbz_map, time_frame, map_id=f"time_{idx}", threshold=35, filter_area=50)
    storms_maps.append(storms_map)

Detecting storms: 100%|██████████| 30/30 [00:54<00:00,  1.82s/it]


## Testing model tracking

In [ ]:
from src.cores.metrics import pod_score, far_score, csi_score

predicted_maps = []
model.processing_map(storms_maps[0])  # Initialize model with the first map

pods = []
fars = []
csis = []

for map in tqdm(storms_maps[1:], desc="Predicting precipitation maps"):
    predicted_map = model.prediction(lead_time=360) # Predict 6 minutes ahead
    model.processing_map(map)  # Update model with the current map

    pods.append(pod_score(map, predicted_map))
    fars.append(far_score(map, predicted_map))
    csis.append(csi_score(map, predicted_map))

Predicting precipitation maps:   3%|▎         | 1/29 [00:00<00:05,  5.54it/s]